In [1]:
# Make sure to update submodule zurcher_data. Otherwise this notebook doesn't work
import yaml
import os
import pandas as pd

from ruspy.estimation.estimation import estimate

In [2]:
import sys # TODO: Extract data functions, such that they can be imported as functions.
sys.path.append('zurcher-data/data/')
from data_processing import data_processing
from data_reading import data_reading

In [3]:
init_dict = {
  'groups': 'group_4',
  'binsize': 5000,
  'beta': 0.9999,
  'states': 90,
  'maint_func': 'linear',
}

In [4]:
data_reading()
data = data_processing(init_dict)

In [5]:
result_transitions, result_fixp = estimate(init_dict, data, repl_4=True)
print(result_transitions, result_fixp)

/home/maxbl/Uni/Master_Thesis/prog/ruspy/ruspy/model_code/fix_point_alg.py:75: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  return ev - np.linalg.lstsq(np.eye(state_size) - t_prime, (ev - iteration_step))[0]


{'state_count': array([[68, 94,  9, ...,  0,  0,  0],
       [ 0, 44, 59, ...,  0,  0,  0],
       [ 0,  0, 47, ...,  0,  0,  0],
       ...,
       [ 0,  0,  0, ...,  1,  1,  0],
       [ 0,  0,  0, ...,  0,  1,  1],
       [ 0,  0,  0, ...,  0,  0,  1]]), 'x': array([0.39189189, 0.59529357, 0.01281454]), 'fun': 3140.5705570938244, 'trans_count': array([1682, 2555,   55])}       fun: 163.58583992638603
 hess_inv: <2x2 LbfgsInvHessProduct with dtype=float64>
      jac: array([-0.00014211,  0.00037801])
  message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 51
      nit: 12
   status: 0
  success: True
        x: array([10.07779269,  2.2941825 ])
